Implementation of "stupid" Thymios which will challenge the "clever" one

Author: Alicia Mauroux, Robotic MA1, Fall 2021

This robot have to follow a lign 
It stops when it sees an object in front of it
When it losts its lign, the idiot Thymio is in "lost" mode

In [3]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

In [27]:
from asgiref.sync import sync_to_async

In [28]:
test_functions = True

In [29]:
#constants
LED = 32

lost = False
w_le = 0
w_ri = 0
last_turn = False #False = left; True = Right
state = 0

Function in order to control the leds. It will tell us if the robot is doing its job or if it's lost. 

In [50]:
@tdmclient.notebook.sync_var
def light_em_up(left=0,right=0):
    global leds_top, leds_buttons, leds_circle, lost, leds_prox_h
    
    leds_prox_h = [0,0,0,0,0,0,0]
    if lost:
        #red
        leds_top = [LED, 0, 0]
    else:
        #yellow
        leds_top = [LED,LED,0]
        leds_circle = [0,0,0,0,0,0,LED,0]
    if left:
        leds_circle = [0,0,0,0,0,0,LED,0]
    elif right: 
        leds_circle = [0,0,LED,0,0,0,0,0]
    else:
        leds_circle = [0,0,0,0,0,0,0,0]
        

In [6]:
@onevent
def button_center():
    global state
    if button_center == 1:
        state = 1 if state==0 else 0

In [31]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    global motor_left_target, motor_right_target
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed

Sensors functions

In [41]:
@tdmclient.notebook.sync_var
def test_ground_white(white_threshold=400, verbose=False):
    """
    Tests whether the two ground sensors have seen white
    param white_threshold: threshold starting which it is considered that the sensor saw white
    param verbose: whether to print status messages or not
    """
    global turn_left, prox_ground_reflected, lost, w_le, w_ri, last_turn
    
    w_le = prox_ground_reflected[0] 
    w_ri = prox_ground_reflected[1] 
    
    #white on the left --> turn right then to keep the track
    if (w_le > white_threshold)&(w_ri < white_threshold):
        turn_left = False
        lost = False
        last_turn = True #Last turn is to the right
        light_em_up(0,1)
        return True
    #white on the right --> turn left then to keep the track
    elif (w_le < white_threshold)&(w_ri > white_threshold):
        lost = False
        turn_left = True
        last_turn = False #Last turn is to the left
        light_em_up(1,0)
        return True
    #white on both direction --> you lost the track! turn back
    elif (w_le > white_threshold)&(w_ri > white_threshold):
        lost = True
        turn_left = False
        light_em_up()
        return True
    else:
        lost = False
        turn_left = False
        light_em_up()
        return False

# Regulators (PI and PD)

In [56]:
@tdmclient.notebook.sync_var
def follow_lign(goal=100):
    global w_le, w_ri, turn_left, lost, last_turn
    MAX_SPEED = 75
    SPEED0 = 200
    TURN = SPEED0//2

    if(test_object()):
        l_speed = 0
        r_speed = 0
    else:
        l_speed = SPEED0
        r_speed = SPEED0

        if (test_ground_white()):
            if(turn_left):
                l_speed = l_speed - TURN
                r_speed = r_speed + TURN
            elif(lost):
                if(last_turn):
                    l_speed = MAX_SPEED
                    r_speed = -MAX_SPEED
                else:
                    l_speed = -MAX_SPEED
                    r_speed = MAX_SPEED
            else:
                l_speed = l_speed  + TURN
                r_speed = r_speed - TURN
    
    return l_speed, r_speed

------------------------------------------------------

In [57]:
async def go_straight(motor_speed=100, white_threshold=600, verbose=False):
    """
    Go Straight Behaviour of the FSM 
    param motor_speed: the Thymio's motor speed
    param white_threshold: threshold starting which it is considered that the ground sensor saw white
    param verbose: whether to print status messages or not
    """
    global prox_ground_reflected, test_ground_white, speed_l, speed_r
    if verbose: print("Starting go straight behaviour")
    
    #l_speed, r_speed = await sync_to_async(PI)()
    #l_speed, r_speed = await sync_to_async(controller)()
    l_speed, r_speed = await sync_to_async(follow_lign)()
    
    
    # Move forward, i.e. set motor speeds
    motors(l_speed, r_speed)

    
    # Until one of the ground sensors sees some white
    saw_white = False
    
    
    while not saw_white:
        test_white = await sync_to_async(test_ground_white)(white_threshold, verbose=verbose)
        if test_white:
            saw_white=True
            if verbose: print("\t Saw white on the ground, exiting go straight behaviour")
        sleep(0.5) #otherwise, variables would not be updated
    return 

In this function we will check is there's an obstacle on the lign the robot is following. If there's one, the robot will stop until the object is removed. 

In [53]:
@tdmclient.notebook.sync_var
def test_object(prox_threshold=2000,prox_threshold_side=1600, verbose=True):
    """
    Tests whether the front proximity sensors saw an object on its way
    param prox_threshold: threshold starting which it is considered that the sensor saw an object
    param verbose: whether to print status messages or not
    """
    global prox_horizontal, leds_circle
    print(prox_horizontal[2], prox_horizontal[1], prox_horizontal[3])
    light_em_up()
    
    if (prox_horizontal[2]>prox_threshold):
        if verbose: print("\t\t Saw a wall")
        leds_circle = [LED,LED,LED,LED,LED,LED,LED,LED]
        return True
    elif (prox_horizontal[1]>prox_threshold_side):
        if verbose: print("\t\t Saw a wall SIDE LEFT")
        leds_circle = [0,LED,LED,LED,0,LED,LED,LED]
        return True
    elif (prox_horizontal[3]>prox_threshold_side):
        if verbose: print("\t\t Saw a wall SIDE RIGHT")
        leds_circle = [0,LED,LED,LED,0,LED,LED,LED]
        return True
    else:
        leds_circle = [0,0,0,0,0,0,0,0]
        return False

# FSM

In [76]:
%%run_python


#constants
LED = 32

lost = False
w_le = 0
w_ri = 0
last_turn = False #False = left; True = Right
state = 0

timer_period[0] = 100   # 100ms sampling time


@onevent
def button_center():
    global state
    if button_center == 1:
        state = 1 if state==0 else 0
        

def light_em_up(left,right):
    global leds_circle, lost
    
    if left:
        leds_circle = [0,0,0,0,0,0,LED,0]
    elif right: 
        leds_circle = [0,0,LED,0,0,0,0,0]
    else:
        leds_circle = [0,0,0,0,0,0,0,0]

def ground_white(white_threshold):
    """
    Tests whether the two ground sensors have seen white
    param white_threshold: threshold starting which it is considered that the sensor saw white
    """
    global turn_left, prox_ground_reflected, lost, w_le, w_ri, last_turn
    
    w_le = prox_ground_reflected[0] 
    w_ri = prox_ground_reflected[1] 
    
    #white on the left --> turn right then to keep the track
    if (w_le > white_threshold):
        if (w_ri < white_threshold):
            turn_left = False
            lost = False
            last_turn = True #Last turn was to the right
            light_em_up(0,1)
            return True
    #white on both direction --> you lost the track! turn back
        else:
            lost = True
            turn_left = False
            light_em_up(1,1)
            return True    
    else:
    #white on the right --> turn left then to keep the track
        if(w_ri > white_threshold):
            lost = False
            turn_left = True
            last_turn = False #Last turn was to the left
            light_em_up(1,0)
            return True
    #following the lign!
        else:
            lost = False
            turn_left = False
            light_em_up(0,0)
            return False

def test_object(prox_threshold,prox_threshold_side):
    """
    Tests whether the front proximity sensors saw an object on its way
    param prox_threshold: threshold starting which it is considered that the sensor saw an object
    """
    global prox_horizontal, leds_circle
    
    if (prox_horizontal[2]>prox_threshold):
        leds_circle = [LED,LED,LED,LED,LED,LED,LED,LED]
        return True
    elif (prox_horizontal[1]>prox_threshold_side):
        leds_circle = [0,LED,LED,LED,0,LED,LED,LED]
        return True
    elif (prox_horizontal[3]>prox_threshold_side):
        leds_circle = [0,LED,LED,LED,0,LED,LED,LED]
        return True
    else:
        leds_circle = [0,0,0,0,0,0,0,0]
        return False
        
def follow_lign(goal):
    global w_le, w_ri, turn_left, lost, last_turn, motor_left_target, motor_right_target
    SPEED0 = 200
    SPEED_LOST = SPEED0//2
    TURN = 150

    if(test_object(2000,1600)):
        l_speed = 0
        r_speed = 0
    else:
        l_speed = SPEED0
        r_speed = SPEED0

        if (ground_white(900)):
            if(turn_left):
                l_speed = l_speed - TURN
                r_speed = r_speed + TURN
            elif(lost):
                if(last_turn):
                    l_speed = SPEED_LOST
                    r_speed = -SPEED_LOST
                else:
                    l_speed = -SPEED_LOST
                    r_speed = SPEED_LOST
            else:
                l_speed = l_speed  + TURN
                r_speed = r_speed - TURN

    motor_left_target = l_speed
    motor_right_target = r_speed

@onevent 
def timer0():
    
    global motor_left_target, motor_right_target
    
    if state:
        follow_lign(100)
    else:
        motor_left_target = 0
        motor_right_target = 0

In [57]:
%%run_python
motor_left_target = 0
motor_right_target = 0

